In [1]:
import ShapeRetrieval: ShapeRetrieval as SR
using ShapeRetrieval: LearnedTimeDiffusionBlock
using SparseArrays
import ShapeRetrieval: Mesh, cot_laplacian, heat_integrator
using LinearAlgebra
using Flux

### Visualization Tools

In [2]:
bunny = SR.load_obj("./meshes/dragon.obj")
bunny = SR.normalize_mesh(bunny)
println("Area: ", sum(bunny.vertex_area))

Area: 1.2018673483165


In [3]:
function spectral_loss(model, x, λ, ϕ, A, y) 
    norm(model(x, λ, ϕ, A) - y)
end

spectral_loss (generic function with 1 method)

## Example Prediction using Spectral Mode

In [4]:
m = LearnedTimeDiffusionBlock(2, :spectral)
heat_signal = zeros(bunny.nv)
heat_signal[[1, 300]] .= 1.0
λ, ϕ, A = SR.get_diffusion_inputs(bunny)
m(heat_signal, λ, ϕ, A)

10194×2 Matrix{Float64}:
 1.78522e-8  7.03035e-8
 1.78522e-8  7.0311e-8
 1.78522e-8  7.03163e-8
 1.78522e-8  7.02986e-8
 1.78522e-8  7.03175e-8
 1.78522e-8  7.03337e-8
 1.78523e-8  7.03731e-8
 1.78523e-8  7.03488e-8
 1.78522e-8  7.03287e-8
 1.74402e-8  2.35475e-8
 ⋮           
 1.73925e-8  4.31515e-8
 1.73925e-8  4.31846e-8
 1.73926e-8  4.33606e-8
 1.73926e-8  4.33298e-8
 1.73926e-8  4.32986e-8
 1.73926e-8  4.32742e-8
 1.73926e-8  4.32377e-8
 1.74221e-8  2.68366e-8
 1.74221e-8  2.68365e-8

#### Sample Training 

In [11]:
data = [(heat_signal, 0.0),]
λ, ϕ, A = SR.get_diffusion_inputs(bunny)

opt_state = Flux.setup(Adam(), m);

@time for i=1:1000
    for d in data
        x, y  = d
        grad = gradient(spectral_loss, m, x, λ, ϕ, A, y)
        Flux.update!(opt_state, m, grad[1])
    end
end
println(m(heat_signal, λ, ϕ, A))

 19.374151 seconds (182.00 k allocations: 93.050 GiB, 15.04% gc time)
0.0003553375294513084


In [6]:
m(heat_signal, SR.get_diffusion_inputs(bunny)...)
println(m.diffusion_time)

(200, 2)


[2.9274950637401447, 0.7993044171182087]


### AD with Implicit Mode

In [7]:
m = LearnedTimeDiffusionBlock(2, :implicit)
Flux.trainable(m::LearnedTimeDiffusionBlock) = (m.diffusion_time,)

function implicit_loss(model, x, L, M, A::Vector{Float64}, y)
    norm(model(x, L, M, A) - y)
end

inputs = SR.get_diffusion_inputs(bunny,:implicit)
m(heat_signal, inputs...)

0.30148325561767775

In [49]:
heat_signal = zeros(bunny.nv)
heat_signal[[1, 300]] .= 1.0
L, M, A = SR.get_diffusion_inputs(bunny, :implicit)
display(m(heat_signal, L, M, A))
data = [(heat_signal, bunny, 0.0),]

opt_state = Flux.setup(Adam(), m)

for i=1:1000
    for d in data
        x, mesh, y  = d
        grad = gradient(implicit_loss, m, x, L,M,A, y)
        Flux.update!(opt_state, m, grad[1])
    end
end

display(m(heat_signal, L, M, A))

0.3014815281221147

0.301481517244275